In [2]:
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth
from datetime import  timedelta, timedelta, datetime

# Extração de dados API

## Lista de DAGS

In [28]:
baseUrl = "http://localhost:8080/api/v1"

username = 'airflow'
password = '1234'

In [29]:
try:
  response = requests.get(baseUrl + "/dags?only_active=true&tags=ativo", auth=HTTPBasicAuth(username, password))
  if response.status_code == 200:
    print("sucesso!")
    data = response.json()
    dagsDF = pd.DataFrame(data['dags'])
    dagsDF = dagsDF[['dag_id', 'dag_display_name', 'is_active', 'is_paused', 'is_subdag', 'root_dag_id', 'owners', 'tags', 'schedule_interval', 'timetable_description']]
except Exception as e:
  print(e)

sucesso!


In [30]:
dagsDF.head()

,dag_id,dag_display_name,is_active,is_paused,is_subdag,root_dag_id,owners,tags,schedule_interval,timetable_description
0,earthquake_data_upstream,Upstream EarthQuake Data,True,False,False,None,[Area 1],"[{'name': 'prd'}, {'name': 'ativo'}]",None,"Never, external triggers only"
1,processa_dados_EarthQuake,Data Count,True,False,False,None,[Area 1],"[{'name': 'prd'}, {'name': 'ativo'}]",None,"Never, external triggers only"
2,teste_execucao,Teste de Execucao,True,False,False,None,[Tester],"[{'name': 'test'}, {'name': 'ativo'}]",None,"Never, external triggers only"
3,teste_falha,Teste de Falha,True,False,False,None,[Tester],"[{'name': 'test'}, {'name': 'ativo'}]",None,"Never, external triggers only"


## DAGS em Execucao...

In [31]:
today = datetime.now()
yesterday = today - timedelta(days=1)
tomorrow = today + timedelta(days=1)
print(yesterday, tomorrow)

2024-10-03 14:36:26.904048 2024-10-05 14:36:26.904048


In [32]:
payload = {
    # "dag_ids": [
    #     "processa_dados_EarthQuake",
    #     "earthquake_data_upstream"
    # ],
    "states": [
        "failed",
        "running",
        "queued",
        "success"
    ],
    "start_date_gte": yesterday.strftime("%Y-%m-%dT00:00:00.000Z"),
    "start_date_lte": tomorrow.strftime("%Y-%m-%dT00:00:00.000Z")
}

In [33]:
try:
  response = requests.post(baseUrl + "/dags/~/dagRuns/list", json=payload, auth=HTTPBasicAuth(username, password))
  if response.status_code == 200:
    print("sucesso!")
    data = response.json()
    runsDF = pd.DataFrame(data['dag_runs'])
    runsDF = runsDF[['dag_id','start_date', 'end_date', 'run_type', 'state']]
    
    runsDF['start_date'] = pd.to_datetime(runsDF['start_date'])
    runsDF['end_date'] = pd.to_datetime(runsDF['end_date'])    
    runsDF['execution_time'] = runsDF['end_date'] - runsDF['start_date']
    
    runsDF['start_date'] = runsDF['start_date'].dt.strftime('%d-%m-%Y %H:%M:%S')
    runsDF['end_date'] = runsDF['end_date'].dt.strftime('%d-%m-%Y %H:%M:%S')
    # runsDF['execution_date'] = runsDF['execution_date'].dt.strftime('%d-%m-%Y %H:%M:%S')
    
except Exception as e:
  print(e)

runsDF.head()

sucesso!


,dag_id,start_date,end_date,run_type,state,execution_time
0,processa_dados_EarthQuake,03-10-2024 12:51:37,03-10-2024 12:51:56,manual,success,0 days 00:00:18.945904
1,earthquake_data_upstream,04-10-2024 15:01:30,04-10-2024 15:01:55,manual,success,0 days 00:00:24.413167
2,processa_dados_EarthQuake,04-10-2024 15:01:31,04-10-2024 15:01:52,manual,success,0 days 00:00:20.984213
3,teste_execucao,04-10-2024 15:02:17,04-10-2024 15:06:23,manual,success,0 days 00:04:06.536070
4,teste_execucao,04-10-2024 15:17:18,04-10-2024 15:21:26,manual,success,0 days 00:04:08.526099


## Join df

In [34]:
df = pd.merge(runsDF, dagsDF, on='dag_id', how='inner')
df['tags'] = df['tags'].apply(lambda x: ', '.join([item['name'] for item in x]))
df['owners'] = df['owners'].apply(lambda x: x[0] if x else '')

df.head()

,dag_id,start_date,end_date,run_type,state,execution_time,dag_display_name,is_active,is_paused,is_subdag,root_dag_id,owners,tags,schedule_interval,timetable_description
0,processa_dados_EarthQuake,03-10-2024 12:51:37,03-10-2024 12:51:56,manual,success,0 days 00:00:18.945904,Data Count,True,False,False,None,Area 1,"prd, ativo",None,"Never, external triggers only"
1,earthquake_data_upstream,04-10-2024 15:01:30,04-10-2024 15:01:55,manual,success,0 days 00:00:24.413167,Upstream EarthQuake Data,True,False,False,None,Area 1,"prd, ativo",None,"Never, external triggers only"
2,processa_dados_EarthQuake,04-10-2024 15:01:31,04-10-2024 15:01:52,manual,success,0 days 00:00:20.984213,Data Count,True,False,False,None,Area 1,"prd, ativo",None,"Never, external triggers only"
3,teste_execucao,04-10-2024 15:02:17,04-10-2024 15:06:23,manual,success,0 days 00:04:06.536070,Teste de Execucao,True,False,False,None,Tester,"test, ativo",None,"Never, external triggers only"
4,teste_execucao,04-10-2024 15:17:18,04-10-2024 15:21:26,manual,success,0 days 00:04:08.526099,Teste de Execucao,True,False,False,None,Tester,"test, ativo",None,"Never, external triggers only"


In [35]:
df.to_csv('./dados_modelo.csv', index=False, header=True)

# Dados extraidos

In [3]:
df = pd.read_csv('./dados_modelo.csv')
df.head()

,dag_id,start_date,end_date,run_type,state,execution_time,dag_display_name,is_active,is_paused,is_subdag,root_dag_id,owners,tags,schedule_interval,timetable_description
0,processa_dados_EarthQuake,03-10-2024 12:51:37,03-10-2024 12:51:56,manual,success,0 days 00:00:18.945904,Data Count,True,False,False,NaN,Area 1,"prd, ativo",NaN,"Never, external triggers only"
1,earthquake_data_upstream,04-10-2024 15:01:30,04-10-2024 15:01:55,manual,success,0 days 00:00:24.413167,Upstream EarthQuake Data,True,False,False,NaN,Area 1,"prd, ativo",NaN,"Never, external triggers only"
2,processa_dados_EarthQuake,04-10-2024 15:01:31,04-10-2024 15:01:52,manual,success,0 days 00:00:20.984213,Data Count,True,False,False,NaN,Area 1,"prd, ativo",NaN,"Never, external triggers only"
3,teste_execucao,04-10-2024 15:02:17,04-10-2024 15:06:23,manual,success,0 days 00:04:06.536070,Teste de Execucao,True,False,False,NaN,Tester,"test, ativo",NaN,"Never, external triggers only"
4,teste_execucao,04-10-2024 15:17:18,04-10-2024 15:21:26,manual,success,0 days 00:04:08.526099,Teste de Execucao,True,False,False,NaN,Tester,"test, ativo",NaN,"Never, external triggers only"
